In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%autosave 0

In [ ]:
import pandas
import utils
from pathlib import Path
import plotly.express as px
import measurements_table
from scipy.interpolate import interp1d

In [ ]:
measurements_table_df = measurements_table.create_measurements_table()

scans_and_sub_measurements_df = pandas.DataFrame()
for root_measurement_name in ['20211116015124_#88_sweeping_laser_DAC']:
    with open(utils.path_to_measurements_directory/Path(root_measurement_name)/Path('scan_1D_sweeping_laser_DAC')/Path('README.txt'), 'r') as ifile:
        for line in ifile:
            if 'This measurement created automatically all the following measurements:' in line:
                continue
            measurement_name = line.replace('\n','')
            scans_and_sub_measurements_df = scans_and_sub_measurements_df.append(
                {
                    'Scan name': measurement_name, 
                    'Laser DAC scan measurement name': root_measurement_name,
                },
                ignore_index = True,
            )
scans_and_sub_measurements_df.set_index('Scan name', inplace=True)
display(scans_and_sub_measurements_df)

Read interpixel distance and calibration factor (previously calculated by the respective scripts) for each measurement:

In [ ]:
interpixel_distances_df = pandas.DataFrame()
for measurement_name in scans_and_sub_measurements_df.index:
    try:
        this_measurement_IPD = utils.read_previously_calculated_inter_pixel_distance(measurement_name)
    except FileNotFoundError:
        this_measurement_IPD = float('NaN')
        print(f'Cannot find IPD for measurement {measurement_name}...')
    try:
        this_measurement_calibration_factor = utils.read_previously_calculated_distance_calibration_factor(measurement_name)
    except FileNotFoundError:
        this_measurement_calibration_factor = float('NaN')
        print(f'Cannot find distance calibration factor for measurement {measurement_name}...')
    try:
        this_measurement_belongs_to_the_laser_DAC_scan = scans_and_sub_measurements_df.loc[measurement_name,'Laser DAC scan measurement name']
    except KeyError:
        this_measurement_belongs_to_the_laser_DAC_scan = '?'
    interpixel_distances_df = interpixel_distances_df.append(
        {
            'Measurement name': measurement_name,
            'IPD (m)': this_measurement_IPD,
            'Distance calibration factor':this_measurement_calibration_factor,
            'Laser DAC scan measurement name': this_measurement_belongs_to_the_laser_DAC_scan,
        },
        ignore_index = True,
    )
interpixel_distances_df.set_index('Measurement name', inplace=True)
display(interpixel_distances_df)

Now build a full dataframe with this information and the information of each detector:

In [ ]:
interpixel_distances_df = interpixel_distances_df.join(measurements_table_df)
interpixel_distances_df.reset_index(inplace=True)
interpixel_distances_df['Device'] = interpixel_distances_df['Measured device']
interpixel_distances_df.set_index('Device', inplace=True)
interpixel_distances_df = interpixel_distances_df.join(utils.bureaucrat.devices_sheet_df)

display(interpixel_distances_df)
print(sorted(interpixel_distances_df.columns))

## Load "Laser DAC" → MIP calibration

In [ ]:
Laser_DAC_scan_on_PIN_diode_df = pandas.read_csv(utils.path_to_measurements_directory/Path('20211007085602_#57_PIN_LaserDACScan/laser_intensity_scan/measured_data.csv'))
Laser_DAC_scan_on_PIN_diode_df = utils.mean_std(
    Laser_DAC_scan_on_PIN_diode_df,
    by = ['n_DAC','DAC value'],
)
ONE_MIP_TO_VOLT_SECOND_RATIO = 35e-12 # V*s/MIP (This comes from the beta scan I did with the PIN didoe.)
DAC_to_MIP = interp1d(
    x = Laser_DAC_scan_on_PIN_diode_df['DAC value'],
    y = Laser_DAC_scan_on_PIN_diode_df['Collected charge (V s) mean']/ONE_MIP_TO_VOLT_SECOND_RATIO,
)

In [ ]:
df = interpixel_distances_df.copy().reset_index()
df['Injected charge (number of MIPs)'] = df['Laser DAC'].apply(DAC_to_MIP)
df['IPD with calibration (m)'] = df['IPD (m)']*df['Distance calibration factor']
df = df.sort_values(by=['Laser DAC','trenches','trench depth'])
fig = utils.line(
    data_frame = df,
    line_group = 'Laser DAC scan measurement name',
    x = 'Injected charge (number of MIPs)',
    y = 'IPD with calibration (m)',
    facet_col = 'wafer',
    facet_row = 'trenches',
    color = 'trench depth',
    symbol = 'pixel border',
    line_dash = 'contact type',
    grouped_legend = True,
    hover_name = 'Measurement name',
    labels = {
        'IPD with calibration (m)': 'IPD (m)',
    },
    text = 'Device',
)
fig.show()
fig.write_html('ipd_vs_injected_charge.html', include_plotlyjs='cdn')